## POC Spatial JOIN NOS data + Cadastre
> éléments non stable à valider pour les données cadastres (besoin de plus de doc).

In [ ]:
import os
import gzip
import shutil
import requests
import geopandas as gpd
from tqdm import tqdm

In [29]:
# 🔹 URL du fichier compressé
url = "https://cadastre.data.gouv.fr/data/etalab-cadastre/2025-01-01/geojson/france/cadastre-france-communes.json.gz"
extension_zip = ".gz"
files = [
    'cadastre-france-sections.json',
    # 'cadastre-france-prefixes_sections.json',
    # 'cadastre-france-communes.json',
    # 'cadastre-france-feuilles.json'
]

In [30]:
def download_file(url, filename):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024  # 1 KB
    progress_bar = tqdm(total=total_size, unit="B", unit_scale=True)

    with open(filename, "wb") as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)

    progress_bar.close()
    print(f"📥 Téléchargement terminé : {filename}")

# ✅ Décompression du fichier .gz
def decompress_gz(input_file, output_file):
    with gzip.open(input_file, "rb") as f_in:
        with open(output_file, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(input_file)
    print(f"📂 Décompression terminée : {output_file}")

In [31]:
for f in files:
    if not os.path.exists(f+extension_zip):
        download_file(url, f+extension_zip)
    if not os.path.exists(f):
        decompress_gz(f+extension_zip, f)

In [32]:
data = gpd.read_file('cadastre-france-sections.json')

In [33]:
data_me = gpd.read_file('clear_cut_processed_2025-02-19.geojson')

In [34]:
data.head()

,id,nom,created,updated,geometry
0,03011,AUROUER,2005-06-06,2022-03-14,"MULTIPOLYGON (((3.29284 46.65822, 3.29285 46.6..."
1,01011,APREMONT,2023-02-21,2024-08-21,"MULTIPOLYGON (((5.64362 46.17778, 5.64395 46.1..."
2,02001,ABBECOURT,2013-12-17,2018-08-06,"MULTIPOLYGON (((3.17881 49.58305, 3.1789 49.58..."
3,04001,AIGLUN,2012-01-17,2021-03-16,"MULTIPOLYGON (((6.14334 44.0416, 6.14357 44.04..."
4,05003,AIGUILLES,2006-12-14,2017-03-17,"MULTIPOLYGON (((6.91207 44.73711, 6.91299 44.7..."


In [35]:
data_me.head()

,id,area_ha,coordonnees,geometry
0,0,0.043094,POINT (423978.93459061446 6666497.349988083),"POLYGON ((3.80855 51.44248, 3.80879 51.44248, ..."
1,1,0.043094,POINT (423945.5689080489 6666473.1329213055),"POLYGON ((3.80831 51.44239, 3.80843 51.44239, ..."
2,2,0.021547,POINT (424012.3002731801 6666335.904164524),"POLYGON ((3.80891 51.44157, 3.80903 51.44157, ..."
3,3,0.021537,POINT (427936.1045428978 6662607.401774076),"POLYGON ((3.84416 51.42064, 3.84428 51.42064, ..."
4,4,0.086147,POINT (428343.16587019834 6662567.059421287),"POLYGON ((3.84775 51.42046, 3.84799 51.42046, ..."


In [36]:
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [37]:
data_me.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [40]:
joined_data = gpd.sjoin(data_me, data, how="inner")

In [41]:
joined_data.head()

,id_left,area_ha,coordonnees,geometry,index_right,id_right,nom,created,updated
81,81,0.021360,POINT (260160.10632990528 6595721.869603097),"POLYGON ((2.337 51.04342, 2.33712 51.04342, 2....",31494,59183,DUNKERQUE,2007-05-21,2021-12-17
82,82,1.516538,POINT (260194.50608997943 6595564.756253276),"POLYGON ((2.33784 51.04287, 2.33844 51.04287, ...",31494,59183,DUNKERQUE,2007-05-21,2021-12-17
113,113,6.123644,POINT (243789.4840940257 6586813.72163727),"POLYGON ((2.19037 50.99393, 2.19049 50.99393, ...",33115,59359,LOON-PLAGE,2007-05-14,2023-11-10
114,114,4.309916,POINT (251006.5454725907 6586614.122269454),"POLYGON ((2.25511 50.99221, 2.25523 50.99221, ...",33115,59359,LOON-PLAGE,2007-05-14,2023-11-10
117,117,1.151821,POINT (241781.05264529577 6584246.701420067),"POLYGON ((2.17239 50.97871, 2.17251 50.97871, ...",25530,59532,SAINT-GEORGES-SUR-L AA,2007-01-15,2024-12-20
